In [ ]:
import pypsa
import pandas as pd
import numpy as np

In [ ]:
solver = "cbc"

*** 
Check-point 1:

**Build a network in PyPSA with two nodes (`bus`) and following components to build a simple green hydrogen production system:**

1. An `Electricity` bus with electricity as `carrier`.
   1. A wind generator inside this bus.
   2. A load_shedding generator inside this bus.
2. A `Hydrogen` bus with hydrogen as `carrier`.
   1. A hydrogen demand load inside this bus.
3. An electrolyser link to convert electricity into hydrogen.

**For simplicity, we assume the hydrogen demand profile to be flat for now. Afterwards, we want to supply electricy by attaching one renewable power plant implemented as (`generator`) (you have to call [`network.set_snapshots`](https://pypsa.readthedocs.io/en/latest/api/_source/pypsa.Network.set_snapshots.html) to select a year). As help you should have a look at the [PyPSA documentation](https://pypsa.readthedocs.io/en/latest/) and the [minimal lopf example](https://pypsa.readthedocs.io/en/latest/examples/lopf-with-heating.html), understand what the [components documentation](https://pypsa.readthedocs.io/en/latest/user-guide/components.html) of PyPSA gives you and that you can find the underlying objective function and constraints in the [LOPF documentation](https://pypsa.readthedocs.io/en/stable/user-guide/optimal-power-flow.html).**


> **Remarks:** For time reasons, you do not have to build the network from scratch. However, to get you acquainted with PyPSA we have omitted a few elements or some of the parameters of the network marked by three question marks `???`. Either, you have to add an element similar to the one in the box above or add a few parameters.

#### Initialize network

In [ ]:
# Create empty PyPSA network
network = pypsa.Network()

In [ ]:
# Set snapshots to the year 2023 and at hourly resolution
snapshots = pd.date_range("01-01-2023", "01-01-2024", freq="H", inclusive="left")
network.set_snapshots(snapshots)

In [ ]:
network.snapshots

In [ ]:
# Import an example of wind daily pattern
wind_pattern = pd.read_csv("../data/weather data/example_onshore_wind_daily_pattern.csv")["daily pattern"]
# annual time-series availability of onshore wind (just a simplified example)
wind_profile = pd.Series(list(wind_pattern)*365, index=network.snapshots)

In [ ]:
wind_profile.head(24).plot()

Add an `electricity` bus with electricity as `carrier`

In [ ]:
network.add(class_name="Bus", name="electricity", carrier="electricity")

Add a `hydrogen` bus with hydrogen as `carrier`

In [ ]:
network.add(class_name="Bus", name="hydrogen", carrier="hydrogen")

In [ ]:
network.buses

Add constant hourly hydrogen load of `100MW` at the hydrogen bus. The name of the load can be `hydrogen_load`

In [ ]:
network.add(class_name="Load", name="hydrogen_load", bus="hydrogen", p_set=100)

In [ ]:
network.loads

Add a wind generator at the electricity bus with a initial capacity of `100MW`, maximum capacity of `500MW`, and marginal cost of `0$/MWh` into the network. The name of the generator can be `onshore_wind`. 

In [ ]:
network.add(
      class_name="Generator", 
      name="onshore_wind",
      bus="electricity",
      carrier="electricity",
      p_nom_extendable=True,
      p_nom=100, # MW
      p_nom_max=500, # MW
      marginal_cost=0, #$/MWh
      p_max_pu=wind_profile
)

Add an electrolyser link, representing the electrolysis conversion with initial capacity of `20MW`, based on provided CAPEX, FOM, VOM, efficiency, interest rate and lifetime.

> **Source:** all costs for the example are taken from [PyPSA technology database](https://github.com/PyPSA/technology-data/blob/master/outputs/costs_2025.csv) and the assumptions in year 2023 in [EU map of hydrogen production costs](https://public.flourish.studio/visualisation/16659363/), with exchange rate of `1.1USD/EUR`

In [ ]:
# Electrolyser's techno-economic parameters are given as:
eur_to_usd = 1.1
lifetime = 10
interest = 0.05

CAPEX = 1590*eur_to_usd # $/MW
FOM = 32*eur_to_usd  # $/MW fixed
VOM = 0 # $/MWh variable
efficiency = 0.65

In [ ]:
# We need to calculate annualized capital expenditure
def calculate_annualised_capex(capex: float, interest: float, lifetime: int):
    crf = (
        interest * (1 + interest) ** lifetime / ((1 + interest) ** lifetime - 1)
    )  # Capital recovery factor
    return capex * crf

In [ ]:
annualized_capex = calculate_annualised_capex(CAPEX, interest, lifetime)

network.add(
    class_name="Link",
    name="electrolyser",
    bus0="electricity",
    bus1="hydrogen",
    p_nom_extendable=True,
    p_nom=20,
    capital_cost=annualized_capex + FOM, #$/MW
    marginal_cost=VOM, #$/MWh
    efficiency=efficiency,
    lifetime=lifetime
)

Now try to solve your network

In [ ]:
# Solve network using cbc solver
network.optimize(pyomo=False, solver_name=solver)

Lets look at the optimisation. Does it solve successfully?

Add a load shedding generator at the electricity bus with marginal cost of `9999$/MWh` (or any large figures) into the network? The name of the generator can be `load_shedding`. 

In [ ]:
# generator for electricity production from load shedding
network.add(
    class_name="Generator",
    name="load_shedding",
    bus="electricity",
    carrier="electricity",
    p_nom_extendable=True,
    marginal_cost=9999,
    )

Now retry to solve your network. Does it solve successfully?

In [ ]:
# Solve network using cbc solver
network.optimize(pyomo=False, solver_name=solver)

Lets look at some results! Back to read the docs. How would you look at results?

In [ ]:
network.generators_t.p[:24*7].plot()

In [ ]:
network.links_t.p0.head()

In [ ]:
network.links_t.p1.head()

In [ ]:
network.loads_t.p.head()

Ending of check-point 1 - export network

In [ ]:
# Export network
network.export_to_netcdf("../results/network_d2_e2-1.nc")

***
Check-point 2:

**Add a solar generator so that we will have two generators + one load-shedding generator in the system**

In [ ]:
# Import check-point 1 network
network = pypsa.Network("../results/network_d2_e2-1.nc")

In [ ]:
# Import an example of solar daily pattern
solar_pattern = pd.read_csv("../data/weather data/example_solar_daily_pattern.csv")["daily pattern"]
# annual time-series availability of solar (just a simplified example)
solar_profile = pd.Series(list(solar_pattern)*365, index=network.snapshots)

In [ ]:
solar_profile.head(24).plot()

Add a solar generator at the electricity bus with a initial capacity of `50MW`, maximum capacity of `500MW`, and marginal cost of `0$/MWh` into the network? The name of the generator can be `solar`. 

In [ ]:
network.add(
      class_name="Generator", 
      name="solar",
      bus="electricity",
      carrier="electricity",
      p_nom_extendable=True,
      p_nom=50,
      p_nom_max=500, # MW
      marginal_cost=0, #$/MWh
      p_max_pu=solar_profile
)

In [ ]:
network.generators

Ending of check-point 2 - Solve network, analyse results and export network

In [ ]:
# Solve network using cbc solver
network.optimize(pyomo=False, solver_name=solver)

In [ ]:
network.generators['p_nom_opt']

In [ ]:
# Inspect the interaction of different power plants to supply loads
network.generators_t.p[:24*7].plot()

In [ ]:
# Inspect the electrolysis conversion
network.links_t.p0.plot()

In [ ]:
# Exporting check-point 2 network
network.export_to_netcdf("../results/network_d2_e2-2.nc")